In [1]:
import numpy as np
import os, sys
sys.path.append('../')
from data_functions import degree2radians, mapping_map_to_sphere, process_df, get_XYZV, create_spherical_heatmap_vals, get_lines
import pandas as pd
df = pd.read_csv("../data/GlobalLandTemperaturesByCity.csv")
df = df[df['AverageTemperature'].notna()]
df['Year'] = np.around([int(dt.split('-')[0]) for dt in df['dt']], -1)
tdf = process_df(df, ['Country'], 'Latitude', 'Longitude')

In [9]:
tdf = df.groupby(['Country', 'Year']).mean().reset_index()
tdf.head()

,Country,Year,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,X,Y,Z
0,Afghanistan,1830,13.092375,2.144333,34.882000,69.842000,0.282603,0.769918,0.571762
1,Afghanistan,1840,13.052596,2.031222,34.535942,68.407167,0.302857,0.764872,0.566749
2,Afghanistan,1850,13.225438,1.829760,34.248273,68.243741,0.306108,0.766607,0.562587
3,Afghanistan,1860,13.067468,1.937233,34.302461,68.141473,0.307298,0.765519,0.563362
4,Afghanistan,1870,13.617368,1.932522,34.158750,68.178750,0.307324,0.767058,0.561281


In [10]:
import pycountry
keys = [list(pycountry.countries)[i].alpha_3 for i in range(len(pycountry.countries))]
names = [list(pycountry.countries)[i].name for i in range(len(pycountry.countries))]
conv = { v:k for k,v in zip(keys,names) }
def match_name(x):
    try:
        return conv[x]
    except KeyError as e:
        for  n in names:
            if x in n:
                return conv[n]
        return 'N/A'
tdf['codes'] = tdf.apply(lambda x: match_name(x['Country']), axis=1)

In [11]:
tdf.head()

,Country,Year,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,X,Y,Z,codes
0,Afghanistan,1830,13.092375,2.144333,34.882000,69.842000,0.282603,0.769918,0.571762,AFG
1,Afghanistan,1840,13.052596,2.031222,34.535942,68.407167,0.302857,0.764872,0.566749,AFG
2,Afghanistan,1850,13.225438,1.829760,34.248273,68.243741,0.306108,0.766607,0.562587,AFG
3,Afghanistan,1860,13.067468,1.937233,34.302461,68.141473,0.307298,0.765519,0.563362,AFG
4,Afghanistan,1870,13.617368,1.932522,34.158750,68.178750,0.307324,0.767058,0.561281,AFG


In [13]:
import plotly.graph_objects as go

years = tdf['Year'].unique()
years.sort()
years_sliders = []

for year in years:
    ttdf = tdf[tdf['Year'] == year]

    years_sliders.append(go.Choropleth(
        visible=False,
        locations = ttdf['codes'],
        z = ttdf['AverageTemperature'],
        text = ttdf['Country'],
        colorscale = 'RdBu',
        reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_ticksuffix = '°C',
        colorbar_title = 'Temperature',
        zmin=tdf['AverageTemperature'].min(), zmax=tdf['AverageTemperature'].max(), zmid=0
    ))

fig = go.Figure(data=years_sliders)

steps = []
for i in range(len(years)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Year: " + str(years[i])}],
              label=str(years[i])  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=len(years),
    currentvalue={"prefix": "Steps: "},
    pad={"t": 50},
    steps=steps,
)]

geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='orthographic'
    )
fig.update_layout(
    sliders=sliders,
    geo=geo
)
fig.data[0].visible = True
fig.write_html('../plots/complex_globe.html')

fig.show()

In [8]:
ttdf

,Latitude,Longitude,Country,Year,X,Y,Z,AverageTemperature,AverageTemperatureUncertainty,codes
1,-52.24,-70.95,Chile,1870,0.199869,-0.578819,-0.790583,5.572269,1.757287,CHL
17,-45.81,-68.08,Argentina,1870,0.260213,-0.646648,-0.717032,10.508537,1.642852,ARG
35,-45.81,169.62,New Zealand,1870,-0.685633,0.125590,-0.717032,7.358602,1.111954,NZL
52,-44.20,172.17,New Zealand,1870,-0.710227,0.097668,-0.697165,10.015694,1.080870,NZL
68,-42.59,-65.45,Argentina,1870,0.305888,-0.669661,-0.676747,12.820241,1.580250,ARG
...,...,...,...,...,...,...,...,...,...,...
29827,65.09,-21.06,Iceland,1870,0.393060,-0.151354,0.906971,1.011546,2.003528,ISL
29855,65.09,24.89,Finland,1870,0.382073,0.177271,0.906971,0.836028,1.079481,FIN
29883,65.09,40.21,Russia,1870,0.321659,0.271919,0.906971,-0.292954,1.477981,RUS
29911,68.31,34.70,Russia,1870,0.303852,0.210397,0.929197,-2.359963,1.300704,RUS
